In [ ]:
# Falcon specification: https://falcon-sign.info/falcon.pdf
# Falcon Python implementation: https://github.com/tprest/falcon.py
# NTRU Solve: https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
# Lecture: https://simons.berkeley.edu/talks/generating-ntru-trapdoors

In [4]:
# global constants
n = 2^9
q=12289
sigma_min = 1.277833697 # see table 3.3 page 51
sigma_max = 1.8205
sigma = 165.736617183
beta = sqrt(34034726)

In [4]:
# use variables consistently
# to avoid clash of types, use variables x, X, y, Y, z, Z as described below
Zx.<x> = PolynomialRing(ZZ) 
#Zphi.<X> = Zx.quotient(x^4+1)
#Qy.<y> = PolynomialRing(QQ)
#Qphi.<Y> = Qy.quotient(y^4+1)
#Zq.<z> = PolynomialRing(Integers(7))
#ZQphi.<Z> = Zq.quotient(z^4+1)

In [22]:
# ============================ POLYNOMIAL UTILS ===================================

# algorythm 16 page 45, author: Maxim Pushkar
def Balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return f.parent()(g)

# formula 3.21 from page 28, author: Evgen Postulga
def Split(f,n):
    f0 = list(f[2*i+0] for i in range(n//2))
    f1 = list(f[2*i+1] for i in range(n//2))
    return f.parent()(f0), f.parent()(f1)  

# formula 3.22 from page 28, author: Evgen Postulga
def Merge(a,b):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a+b

# formula 3.6 from page 23, author: Evgen Postulga
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    # return p.parent()(f)
    return Zx(f) 

# formula 3.10 from page 24, author: Evgen Postulga
def InnerProduct(a,b,n):
    # return sum([a[i] * b[i] for i in range(n)])
    s=0
    for i in range(n):
        s=s+a[i]*b[i]
    return(s)

# formula 3.9 from page 24, author: Evgen Postulga
def EuclideanNorm(a,n):
    # return sqrt(InnerProduct(a,a,n))
    b=InnerProduct(a,a,n)
    return b^(1/2)

# formula 3.25 from page 30, author: Maxim Pushkar
def FieldNorm(f, n):
    f0, f1 = split(f, n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n/2)+1)
    # return (f0^2-x*f1^2)%(x^(n/2)+1)

# NNT representation of polynomial f from Zq[x]/x^n+1, page 28, author Karina Ilchenko
def NTT(f, n, q):
    # Zp
    #g = f.change_ring(Integers(q))
    #return list(g(root[0]) for root in (x^n+1).roots(Integers(q)))    
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans


In [1]:
# ======================== Lattice Matrices ============================

# author: Evgen Postulga
def CyclicRotate(input, n):
    #return input[(len(input)-n):]+input[:(len(input)-n)]
    return input[-n:] + input[0:-n]

# author: Evgen Postulga
def PolyToCirculant(p, n):
    M=[]
    k=p.coefficients(sparse=False)
    while len(k)!=n:
        k.append(0)
    for i in range(n):
        m = CyclicRotate(k, i)
        M.append(m)
    return Matrix(M)

# author: Evgen Postulga
def CirculantToPoly(M):
    # Zx.<x> = PolynomialRing(ZZ)
    return Zx(list(M[0]))
    
# author: Evgen Postulga
def PolyToLattice4(p00, p01, p10, p11, n):
    M=[]
    p=[p00.coefficients(sparse=False),p01.coefficients(sparse=False),p10.coefficients(sparse=False),p11.coefficients(sparse=False)]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return Matrix(M)

# author: Evgen Postulga
def LatticeToPoly4(M, n_2):
    p00=M[0][:n]
    p01=M[0][n:]
    p10=M[n][:n]
    p11=M[n][n:]
    return p00, p01, p10, p11

# formula 3.7 page 23, author: Evgen Postulga
def HermitianAdjointMatrix(M, n2):
    a, c, b, d = LatticeToPoly4(M, n2/2)
    a, c, b, d = HermitianAdjointPoly(a, n2/2), HermitianAdjointPoly(c, n2/2),HermitianAdjointPoly(b, n2/2),HermitianAdjointPoly(d, n2/2)
    return PolyToLattice4(a, c, b, d, n2/2)

In [ ]:
# =========================== RANDOM GENERATORS ============================

# formula 3.32 on page 40, author: Aleksey Mokhonko
def UniformBits(k):
    return bytes(list(floor(uniform(0, 256)) for i in range(k / 8)))

# algorythm 12 page 41, author: Dmytro Husan
def BaseSampler():
    u = UniformBits(72)
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

# algorythm 13 page 42, author: Dmytro Husan
def ApproxExp(x, ccs):
    C = (0x00000004741183A3, 0x00000036548CFC06, 0x0000024FDCBF140A, 0x0000171D939DE045,
         0x0000D00CF58F6F84, 0x000680681CF796E3, 0x002D82D8305B0FEA, 0x011111110E066FD0,
         0x0555555555070F00, 0x155555555581FF00, 0x400000000002B400, 0x7FFFFFFFFFFF4800, 0x8000000000000000)
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y
    
# algorithm 14 page 43, author: Dmytro Husan
def BerExp(x, ccs):
    s = floor(x/ln(2))
    r = x - s*ln(2)
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = int.from_bytes(urandom(3), "little")
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

# algorithm 15, page 43, author: Dmytro Husan
def SamplerZ(mu, sigma, sigma_min, sigma_max):
    r = mu - floor(mu)
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + floor(mu)

In [ ]:
# ============================ GENERATE f, F, g, G =============================

# algorithm 7, page 35, author Karina Ilchenko
# also https://www.h2020prometheus.eu/sites/default/files/2019-11/More%20Efficient%20Algorithms%20for%20the%20NTRU%20Key%20Generation%20using%20the%20Field%20Norm.pdf 
# page 10
def Reduce(f, g, F, G, n):
    
    T = Zx.change_ring(QQ).quotient(x^n+1) 
    
    iks = f.parent()([0, 1])
    f_star = HermitianAdjointPoly(f, n)
    g_star = HermitianAdjointPoly(g, n)
    while True:
        num = F*f_star + G*g_star
        num = T(num)
        den = f*f_star + g*g_star
        den = 1 / T(den)
        res = num * den
        k = Zx([int(round(elt)) for elt in res])
        F = F - k*f 
        G = G - k*g
        if all(elt == 0 for elt in k):
            break

    return f, g, F, G

# algorithm 6, page 35, author Karina Ilchenko
# also https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
# and https://www.h2020prometheus.eu/sites/default/files/2019-11/More%20Efficient%20Algorithms%20for%20the%20NTRU%20Key%20Generation%20using%20the%20Field%20Norm.pdf
def NTRUSolve(f, g, n, q):
    if n == 1:
        # u, v are numbers
        gcd_, u, v = xgcd(f[0], g[0])
        #print("gcd", u * f + v * g, "\n")
        if gcd_ != 1:
            return None, None, False
        F, G = -v*q, u*q
        #print("F1, G1", F / q, G / q)
        return F, G, True
    else:
        # ▷ f′, g′, F′, G′ ∈ Z[x]/(x^n/2 + 1)
        # ▷ N as defined in either (3.25) or (3.26)
        f_ = FieldNorm(f, n) 
        g_ = FieldNorm(g, n) 
        #print(n//2, f_, g_, sep="\n")
        F_, G_, flag = NTRUSolve(f_, g_, n//2, q)
        if flag:
            F = F_.subs(x=x^2) * g.subs(x=-x) 
            G = G_.subs(x=x^2) * f.subs(x=-x)
            #print("F, G", F, G, sep="\n")
            f, g, F, G = Reduce(f, g, F, G, n)
            return F % (x^n +1), G % (x^n +1), flag
        else:
            return F_, G_, flag

# algorithm 6, page 35, author Karina Ilchenko
# also https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf
def NTRUGen(q, n):
    
    def gen_poly(n, q):
        
        def D(mu=0):
            z = 0
            for i in range(1, 4096/n + 1):
                sigma_star = 1.17 * sqrt(q / 8192)
                sigmamin, sigmamax = 1.277833697, 1.8205
                zi = SamplerZ(mu, sigma_star, sigmamin, sigmamax)
                z += zi
            return z

        f = [0] * n
        g = [0] * n
        for i in range(n):
            f[i] = D()
            g[i] = D()
        f = Zx(f) % phi
        g = Zx(g) % phi
        return f, g
    
    def gs_norm(f, g, q, n):
        T = Zx.change_ring(QQ).quotient(x^n+1) 
        # Using (3.9) with (3.8) or (3.10)    
        f_star = HermitianAdjointPoly(f, n)
        g_star = HermitianAdjointPoly(g, n)
        first = EuclideanNorm([*g.coefficients(sparse=False), *(-f).coefficients(sparse=False)], n)
        s1 = (q * T(f_star)) / T((f*f_star + g*g_star))
        s2 = (q * T(g_star)) / T((f*f_star + g*g_star))
        second = EuclideanNorm(list(s1) + list(s2), n)
        gamma = max(first, second)
        return gamma
    
    
    while True:
        while True:
            while True:

                f, g = gen_poly(n, q)
                # print(f, g, sep="\n")

                if gs_norm(f, g, q, n) > (1.17 ** 2) * q:
                    # print("restart norm\n")
                    continue
                break

            if  0 in NTT(f, n, q):
                # print("restart ntt\n")
                continue
            break
                
        F, G, flag = NTRUSolve(f, g, n, q)
        
        if not flag:
            # print("restart solve")
            continue
        else:
            F, G = F % (x^n +1), G % (x^n +1)
            F = Zx([int(coef) for coef in F.coefficients(sparse=False)])
            G = Zx([int(coef) for coef in G.coefficients(sparse=False)])
            print("(f*G - g*F) % (x^n + 1) == q", (f*G - g*F) % (x^n + 1) == q)
            break
            
    return f, g, F, G    

In [26]:
# ============================ FAST FOURIER TRANSFORM =========================
# Accordint to page 27, polynomials must be from Q[y]/(y^n+1)

# roots of monic polynomial https://github.com/tprest/falcon.py/blob/master/scripts/generate_constants.sage
def Omega(n):
    #if n == 2:
    #    return [I, -I]
    #else:
    #    return sum([[sqrt(elt), - sqrt(elt)] for elt in Omega(n // 2)], [])
    phi4 = cyclotomic_polynomial(4)
    phi_n = phi4.complex_roots()
    phi_n.reverse()
    k = 2
    while k != n:
        phi_n = sum([[sqrt(elt), - sqrt(elt)] for elt in phi_n], [])
        k = 2*k
    return phi_n

# formula 3.18, page 27, author: Evgen Postulga
def FFT(f):
    n = len(list(f))
    return [lift(f).subs(root) for root in Omega(n)]

# algorithm 1, page 29, author: Evgen Postulga
def splitfft(f):
    n = len(f)
    w = Omega(n)
    f0 = [0] * (n // 2)
    f1 = [0] * (n // 2)
    for i in range(n // 2):
        f0[i] = 0.5 * (f[2 * i] + f[2 * i + 1])
        f1[i] = 0.5 * (f[2 * i] - f[2 * i + 1]) / w[2 * i]
    return f0, f1

# algorithm 2, page 29, author: Evgen Postulga
def mergefft(f0, f1):
    n = 2 * len(f0)
    w = Omega(n)
    f = [0] * n
    for i in range(n // 2):
        f[2 * i + 0] = f0[i] + w[2 * i] * f1[i]
        f[2 * i + 1] = f0[i] - w[2 * i] * f1[i]
    return f

# https://github.com/tprest/falcon.py/blob/master/fft.py
def invFFT(f_fft):
    n = len(f_fft)
    if (n > 2):
        f0_fft, f1_fft = splitfft(f_fft)
        f0 = invFFT(f0_fft)
        f1 = invFFT(f1_fft)

        f = n*[0]
        for i in range(n//2):
            f[2*i+0] = f0[i]
            f[2*i+1] = f1[i] 

    elif (n == 2):
        f = [0, 0]
        f[0] = f_fft[0].real()
        f[1] = f_fft[0].imag()
    return f

def mul_fft(a,b):
    return list(a[i] * b[i] for i in range(len(a)))
def div_fft(a,b):
    return list(a[i] / b[i] for i in range(len(a)))
def add_fft(a,b):
    return list(a[i] + b[i] for i in range(len(a)))
def sub_fft(a,b):
    return list(a[i] - b[i] for i in range(len(a)))
def neg_fft(a):
    return list(     - a[i] for i in range(len(a)))

In [19]:
# FFT Test 1: test that FFT and invFFT are inverse operations
Qy.<y> = PolynomialRing(QQ)
Qphi.<Y> = Qy.quotient(y^4+1) 
f = Y^3-5*Y+1; print(f)
g = invFFT(FFT(f)); print(Qphi(g))

Y^3 - 5*Y + 1
Y^3 - 5*Y + 1


In [29]:
# FFT Test 2: operations +-*/ over polynomials correspond to per coordinate +-*/
f = Y^3-5*Y+1; g = Y^2-7*Y+3
print("Multiplication: ", mul_fft(FFT(f), FFT(g)), " ===", FFT(f * g))
print("Division: ",       div_fft(FFT(f), FFT(g)), " ===", FFT(f / g))
print("Addition: ",       add_fft(FFT(f), FFT(g)), " ===", FFT(f + g))
print("Subtraction: ",    sub_fft(FFT(f), FFT(g)), " ===", FFT(f - g))
print("Negation: ",       neg_fft(FFT(f)),         " ===", FFT(-f))

Multiplication:  [-4.84924240491749 + 18.3223304703363*I, 24.8492424049175 + 53.6776695296637*I, -4.84924240491749 - 18.3223304703363*I, 24.8492424049175 - 53.6776695296637*I]  === [-4.84924240491750 + 18.3223304703363*I, 24.8492424049175 + 53.6776695296637*I, -4.84924240491750 - 18.3223304703363*I, 24.8492424049175 - 53.6776695296637*I]
Division:  [0.901653699819546 - 0.375883187601447*I, 0.593380278225410 - 0.0883091804069174*I, 0.901653699819546 + 0.375883187601447*I, 0.593380278225410 + 0.0883091804069174*I]  === [0.901653699819545 - 0.375883187601446*I, 0.593380278225410 - 0.0883091804069174*I, 0.901653699819545 + 0.375883187601446*I, 0.593380278225410 + 0.0883091804069174*I]
Addition:  [-5.19238815542512 - 6.77817459305202*I, 13.1923881554251 + 8.77817459305202*I, -5.19238815542512 + 6.77817459305202*I, 13.1923881554251 - 8.77817459305202*I]  === [-5.19238815542512 - 6.77817459305202*I, 13.1923881554251 + 8.77817459305202*I, -5.19238815542512 + 6.77817459305202*I, 13.192388155425

In [30]:
# ========================== SIGN / VERIFY ==============================

# algorithm 3 page 31, author: Maxim Pushkar
# pip3 install pycryptodome
from Crypto.Hash import SHAKE256
def HashToPoint(noise, message, q, n):
    k = floor(2^16 / q)
    c = n*[0]
    ctx = SHAKE256.new()
    ctx.update(noise)
    ctx.update(message)
    i = 0
    while i < n:
        t = int.from_bytes(ctx.read(2), byteorder='big')
        if t < k * q:
            c[i] = t % q
            i += 1
    return c

# algorithm 11 page 40
def ffSampling(t0, t1, T, sigma_min, sigma_max, q, n):
    pass

# algorithm 10, page 39
def Sign(message, B, T, bound, sigma_min, sigma_max, q, n):
    pass

# algorithm 16 page 45 
def Verify(message, r, s2, pk, beta, q, n):
    pass